In [ ]:
from adam_impact_study.analysis.collect import collect_all_results, collect_all_window_results
from adam_impact_study.analysis.main import run_all_analysis, summarize_impact_study_results
from adam_impact_study.analysis.plots import make_analysis_plots
import pathlib
from adam_impact_study.types import ImpactorOrbits, Observations, ResultsTiming, WindowResult, ImpactorResultSummary

In [2]:
run_dir = "/Users/natetellis/data/mar_run/results/"
out_dir = "/Users/natetellis/data/mar_run_analysis/"

In [3]:
# impactor_orbits, observations, timings, window_results = collect_all_results(run_dir)

In [4]:
# write out these tables
# impactor_orbits.to_parquet("/Users/natetellis/data/mar_run_analysis/impactor_orbits.parquet")
# observations.to_parquet("/Users/natetellis/data/mar_run_analysis/observations.parquet")
# timings.to_parquet("/Users/natetellis/data/mar_run_analysis/results_timing.parquet")
# window_results.to_parquet("/Users/natetellis/data/mar_run_analysis/window_results.parquet")



In [5]:
# Now load from them
impactor_orbits = ImpactorOrbits.from_parquet("/Users/natetellis/data/mar_run_analysis/impactor_orbits.parquet")
observations = Observations.from_parquet("/Users/natetellis/data/mar_run_analysis/observations.parquet")
results_timing = ResultsTiming.from_parquet("/Users/natetellis/data/mar_run_analysis/results_timing.parquet")
window_results = WindowResult.from_parquet("/Users/natetellis/data/mar_run_analysis/window_results.parquet")



In [6]:
# summary_results = summarize_impact_study_results(
#     impactor_orbits, observations, results_timing, window_results, out_dir,
# )

In [7]:
# write out study results
# summary_results.to_parquet("/Users/natetellis/data/mar_run_analysis/summary_backup.parquet")
# load the summary results
summary_results = ImpactorResultSummary.from_parquet("/Users/natetellis/data/mar_run_analysis/summary_backup.parquet")


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

summary_results.to_dataframe()

In [ ]:
make_analysis_plots(summary_results,window_results, out_dir)